## Creating a script to parse blast files of custom or standard tabular format

**Started with standard function to parse blast files of set format**

In [68]:
# function for parsing blast file
def old_blast_parser(blastfile, tab=None):
    """parse tabular blast files to retrieve all information for downstream"""
    blast = open(blastfile)
    if tab is None: #'standard': # alternative add later
        std = {'qseqid':0, 'sseqid':1, 'pident':2, 'length':3, 'mismatch':4, 'gapopen':5,'qstart':6, 'qend':7, 'sstart':8, 'send':9, 'evalue':10, 'bitscore':11}
        fmt_list = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
    #else:
    integers = ['qlen','slen','qstart','qend','sstart','send','length','nident','mismatch','positive','gapopen','gaps','qcovs','qcovhsp']    
    floaters = ['bitscore','score','pident','ppos']
    ranges = {}
    rng_index = {}
    #len_min = 100
    #min_pct = 30
    hit_index = {}
    hit_dict = {}
    hitlist = []
    fld_range = len(fmt_list)
    field_dict = {}
    for f in range(0,fld_range):
        field_dict[fmt_list[f]]=f
        
    ct = 0
    for hit in blast:
        hit = hit.strip('\n')
        parts = hit.split('\t')
        ## have to change this to be flexible as to input format, start with standard format
        
        for fld in range(0,fld_range):
            fld_name = fmt_list[fld]
            fld_value = field_dict[fld_name]
            ranges.setdefault(hitnum, {})['qseqid']=query
            
        query = parts[std['qseqid']]
        #print(query)
        subject = parts[std['sseqid']]
        pct_ident = float(parts[std['pident']])
        aln_length = int(parts[std['length']])
        num_mismatch = int(parts[std['mismatch']])
        gap_open = int(parts[std['gapopen']])
        query_start = int(parts[std['qstart']])
        query_end = int(parts[std['qend']])
        substart = int(parts[std['sstart']])
        subend = int(parts[std['send']])
        e_val = parts[std['evalue']]
        bit_score = parts[std['bitscore']]
        #if pct_ident > min_pct: # filtering will happen outside script
        if query in hitlist:
            ct += 1
        else:
            hitlist.append(query)
            ct = 0
        hitnum = query+';hit'+str(ct)
        rng_index.setdefault(query, []).append(hitnum)
        ranges.setdefault(hitnum, {})['qseqid']=query
        ranges.setdefault(hitnum, {})['sseqid']=subject
        ranges.setdefault(hitnum, {})['pident']=pct_ident
        ranges.setdefault(hitnum, {})['length']=aln_length
        ranges.setdefault(hitnum, {})['mismatch']=num_mismatch
        ranges.setdefault(hitnum, {})['gapopen']=gap_open
        ranges.setdefault(hitnum, {})['qstart']=query_start
        ranges.setdefault(hitnum, {})['qend']=query_end
        ranges.setdefault(hitnum, {})['sstart']=substart
        ranges.setdefault(hitnum, {})['send']=subend
        ranges.setdefault(hitnum, {})['evalue']=e_val
        ranges.setdefault(hitnum, {})['bitscore']=bit_score
    return(rng_index, ranges)


**The parsing function below will handle any custom tabular format**

In [23]:
# function for parsing blast file, default is standard tab format
def blast_parser(blastfile, tab='standard'):
    """parse tabular blast files to retrieve all information for downstream"""
    blast = open(blastfile)
    if tab is 'standard': #'standard': # alternative add later 
        fmt_list = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
    else:
        fmt_list = tab
    integers = ['qlen','slen','qstart','qend','sstart','send','length','nident','mismatch','positive','gapopen','gaps','qcovs','qcovhsp']    
    floaters = ['bitscore','score','pident','ppos','evalue']
    ranges = {}
    rng_index = {}
    hit_index = {}
    hit_dict = {}
    hitlist = []
    # adding list of queries to pass to tuple, to preserve order of queries
    query_list = []
    hit_order = []
    fld_range = len(fmt_list)
    ct = 0
    for hit in blast:
        hit = hit.strip('\n')
        parts = hit.split('\t')
        for fld in range(0,fld_range):
            fld_name = fmt_list[fld]
            # differentiate between field types
            if fld_name in integers:
                fld_value = int(parts[fld])
            elif fld_name in floaters:
                fld_value = float(parts[fld])
            else:
                fld_value = parts[fld]    
            if fld == 0:    
                if fld_value in hitlist:
                    ct += 1
                else:
                    hitlist.append(fld_value)
                    ct = 0
                hitnum = str(fld_value)+';hit'+str(ct)
                rng_index.setdefault(str(fld_value), []).append(hitnum)
                if fld_value not in query_list:
                    query_list.append(fld_value)

            ranges.setdefault(hitnum, {})[fld_name]=fld_value
            if hitnum not in hit_order:
                hit_order.append(hitnum)
            

    return(rng_index, ranges, query_list, hit_order)


In [24]:
## testing with a nonstandard format
std_fmt = ['qseqid', 'sseqid', 'length', 'pident', 'mismatch', 'qcovs', 'evalue', 'bitscore', 'qstart', 'qend', 'sstart', 'send', 'gapopen']

In [25]:
blast_results = blast_parser('example_blastn_custom_output.outfmt6', std_fmt)

In [26]:
# returned set with two dictionaries: first has a list of hits for each query
blast_results[0]['MA_10046850g0010']

['MA_10046850g0010;hit0',
 'MA_10046850g0010;hit1',
 'MA_10046850g0010;hit2',
 'MA_10046850g0010;hit3',
 'MA_10046850g0010;hit4']

In [18]:
# the second dictionary has all information for each hit
blast_results[1]['MA_10429268g0010;hit0']

{'bitscore': 3219.0,
 'evalue': 0.0,
 'gapopen': 1,
 'length': 1780,
 'mismatch': 11,
 'pident': 99.326,
 'qcovs': 100,
 'qend': 1779,
 'qseqid': 'MA_10429268g0010',
 'qstart': 1,
 'send': 4241,
 'sseqid': 'Pg-02r141203s3085282',
 'sstart': 2462}

In [27]:
query_ls = blast_results[3]
query_ls[1]

'MA_10046850g0010;hit1'

In [7]:
# write script to filter blast_parser output to file as table
def get_parameters(parameter_file):
    para_file = open(parameter_file, 'r')
    para_dict = {}
    for line in para_file:
        line = line.strip('\n')
        if line.startswith('#'):
            continue
        else:
            line_parts = line.split('=')
            para_name = line_parts[0]
            para_value =line_parts[1]
            if para_name == 'FORMAT':
                fieldlist = para_value.split(' ')
                para_dict['FORMAT']=fieldlist
            elif para_name == 'MIN_LENGTH':
                para_dict['MIN_LENGTH']=int(para_value)
            elif para_name == 'MIN_PCT_ID':
                para_dict['MIN_PCT_ID']=int(para_value)
            elif para_name == 'QCOV':
                para_dict['QCOV']=int(para_value)
            elif para_name == 'MAX_MISMATCH':
                para_dict['MAX_MISMATCH']=int(para_value)
            
    
    
    return para_dict

In [6]:
paras1 = get_parameters('parameters_file_template.txt')
paras1

{'FORMAT': ['qseqid',
  'sseqid',
  'length',
  'pident',
  'mismatch',
  'qcovs',
  'evalue',
  'bitscore',
  'qstart',
  'qend',
  'sstart',
  'send',
  'gapopen'],
 'MIN_LENGTH': 500,
 'MIN_PCT_ID': 95}

In [16]:
fmt1 = paras1['FORMAT']
fmt1

['qseqid',
 'sseqid',
 'length',
 'pident',
 'mismatch',
 'qcovs',
 'evalue',
 'bitscore',
 'qstart',
 'qend',
 'sstart',
 'send',
 'gapopen']

In [30]:
# a script that implements the blast parser, filters the result for length and pident, and outputs to summary

blastout = open('blast_results_filtered.txt', 'w')
blastfile = ('example_blastn_custom_output.outfmt6')
parameter_file = ('parameters_file_template.txt')

blast_parameters = get_parameters(parameter_file)
blast_tab_format = blast_parameters['FORMAT']
min_length = blast_parameters['MIN_LENGTH']
min_pct = blast_parameters['MIN_PCT_ID']

blast_results = blast_parser(blastfile, blast_tab_format)

query_index = blast_results[0]
hit_results = blast_results[1]

for key, value in query_index.iteritems():
    for hit in value: # now loops through lists of hits for this query
        hit_dict = hit_results[hit]
        if hit_dict['length'] > min_length:
            if hit_dict['pident'] > min_pct:
                # if it passes, write all fields to new file
                # use blast_tab_format to put values in order
                blaststring = ''
                for field in blast_tab_format:
                    #print field
                    field_value = str(hit_dict[field])
                    blaststring = blaststring+field_value+'\t'
                blastwrite = blaststring+'\n'
                blastwrite = blastwrite.replace('\t\n','\n')
                blastout.write(blastwrite)
                    
blastout.close()

have to set it up so all conditions can be tested simultaneously. Not nested 'if' conditions, but test each one, and add them up (if x > minimum: count += 1 or something), then total of all condition tests must add up to number of conditions from parameters

To have a variable number of conditions to test to filter blast results: create dictionary from parameters file, and can use length of dictionary to set number of conditions

In [4]:
# match the parameter to blast output fields
field_match = {'MIN_LENGTH':'length','MIN_PCT_ID':'pident','MAX_MISMATCH':'mismatch','EVALUE':'evalue','SUBJECT_ID':'sseqid'}

In [8]:
# a script that implements the blast parser, filters the result for length and pident, and outputs to summary
# modified
# establish how to test conditions
mins = ['MIN_LENGTH','MIN_PCT_ID']
maxes = ['MAX_MISMATCH','EVALUE']
matches = ['SUBJECT_ID']

blastout = open('blast_results_filtered_newer.txt', 'w')
blastfile = ('example_blastn_custom_output.outfmt6')
parameter_file = ('parameters_file_template.txt')

blast_parameters = get_parameters(parameter_file)
# get format
blast_tab_format = blast_parameters['FORMAT']
# take out format for filtering
blast_parameters.pop('FORMAT')
# get number of conditions to test
num_conditions = len(blast_parameters)
# get blast results
blast_results = blast_parser(blastfile, blast_tab_format)

query_index = blast_results[0]
hit_results = blast_results[1]

for key, value in query_index.iteritems():
    for hit in value: # now loops through lists of hits for this query
        conditions_reached = 0
        hit_dict = hit_results[hit]
        # now test all conditions separately
        for k,v in blast_parameters.iteritems():
            blast_field = field_match[k]
            if k in mins:
                if hit_dict[blast_field] > v:
                    conditions_reached += 1
            elif k in maxes:
                if hit_dict[blast_field] < v:
                    conditions_reached += 1
            elif k in matches:
                if hit_dict[blast_field] == v:
                    conditions_reached += 1
 
        if conditions_reached == num_conditions:
            # use blast_tab_format to put values in order
            blaststring = ''
            for field in blast_tab_format:
                #print field
                field_value = str(hit_dict[field])
                blaststring = blaststring+field_value+'\t'
            blastwrite = blaststring+'\n'
            blastwrite = blastwrite.replace('\t\n','\n')
            blastout.write(blastwrite)
                    
blastout.close()
print(blast_parameters)

{'MIN_LENGTH': 500, 'MAX_MISMATCH': 20, 'MIN_PCT_ID': 95}


In [11]:
dict1 = {'a':1,'b':2,'c':3}
dict2 = dict1.pop('c')
dict1

{'a': 1, 'b': 2}

In [9]:
a = 1.222
b = 1.223
b > a

True

### Rewrite the parameter filter as a function

In [10]:
def filter_blast(blast_result_dict, blast_parameters):
    """Filter a blast result dictionary based on variable parameters"""
    mins = ['MIN_LENGTH','MIN_PCT_ID']
    maxes = ['MAX_MISMATCH','EVALUE']
    matches = ['SUBJECT_ID']
    filtered_blast = {}
    blast_parameters.pop('FORMAT')
    num_conditions = len(blast_parameters)
    query_index = blast_result_dict[0]
    hit_results = blast_result_dict[1]
    for key, value in query_index.iteritems():
        for hit in value: # now loops through lists of hits for this query
            conditions_reached = 0
            hit_dict = hit_results[hit]
            # now test all conditions separately
            for k,v in blast_parameters.iteritems():
                blast_field = field_match[k]
                if k in mins:
                    if hit_dict[blast_field] > v:
                        conditions_reached += 1
                elif k in maxes:
                    if hit_dict[blast_field] < v:
                        conditions_reached += 1
                elif k in matches:
                    if hit_dict[blast_field] == v:
                        conditions_reached += 1

            if conditions_reached == num_conditions:
                filtered_blast[hit]=hit_dict
    return filtered_blast

In [29]:
# run script from functions

blastout = open('blast_results_filtered_from_function2.txt', 'w')
blastfile = ('example_blastn_custom_output.outfmt6')
parameter_file = ('parameters_file_template.txt')

blast_parameters = get_parameters(parameter_file)
# get format
blast_tab_format = blast_parameters['FORMAT']
# have to put default format parameters here, in case none provided

# get blast results
blast_results = blast_parser(blastfile, blast_tab_format)
hit_order = blast_results[3]
# filter blast results
filt_blast_results = filter_blast(blast_results, blast_parameters)

# write filtered results to file
for hit in hit_order:
    if hit in filt_blast_results:
        hit_values = filt_blast_results[hit]
        #for key,value in filt_blast_results.iteritems():
        blaststring = ''
        for field in blast_tab_format:
            #print field
            field_value = str(hit_values[field])
            blaststring = blaststring+field_value+'\t'
        blastwrite = blaststring+'\n'
        blastwrite = blastwrite.replace('\t\n','\n')
        blastout.write(blastwrite)
                    
blastout.close()

In [28]:
# just try to order the dictionary, so they all come out 
#filt_blast_results
